In [2]:
import json
import pandas as pd
import sys
import spacy
import os


sys.path.insert(1, 'C:/Users/tom/projects/skill-skeleton/utils/')
sys.path.insert(2, 'C:/Users/tom/projects/skill-skeleton/utils/neo4j/')
import kb_util

from connection import Neo4jConnection
import query as query

def search_matching_skill(nlp, found_skill, skills, threshold=0.9):    
    
    search = nlp(found_skill)    
    high_score = 0
    term = ""

    for elem in skills:
        match = nlp(elem)            
        score = search.similarity(match)
        
        if score == 1.0:
            return match, score
        
        if score > high_score:            
            print(search, "<->", match, "==", score)
            high_score = score
            term = match
    
    if high_score > threshold:
        return term, high_score
    else:
        return None, 0  


def clean_skill_list(found_skills, skills, nlp, threshold):   
    
    found_skills = [x.lower() for x in found_skills]
    skills = [x.lower() for x in skills]    
    new_set= set()
    
    for found_skill in found_skills:    
   
        if found_skill in skills:
            #print("Found in KB", found_skill)           
            new_set.add(found_skill)
            continue
        
        term, high_score = search_matching_skill(nlp, found_skill, skills, threshold)
        if high_score > 0:
            if high_score == 1.0:
                #print("Found in KB", found_skill)           
                new_set.add(found_skill)
            else:
                #print(search, "<->", term, "==", high_score)              
                new_set.add(term.text)
        else:
            new_set.add(found_skill)  
    
    return list(new_set)


##############################################################################################################################


def extract_tags(row):
    info = row['ENTITIES']
    new_list = []
    for i in info['entities']:
        new_list.append(row['TEXT'][i[0]:i[1]])
    return new_list


def extract_tags_from_annotation(input_file, source_file, output_file, delimiter):
    # Open the JSON file
    with open(input_file) as f:
        # Load the JSON data
        data = json.load(f)

    df_annotated = pd.DataFrame(data['annotations'], columns=['TEXT', 'ENTITIES'])

    df_source = pd.read_csv(source_file,delimiter=delimiter)

    if len(df_annotated) != len(df_source):
        print('The number of rows in the source file and the annotated file are different')
        exit(1)

    df_source['skills'] = df_annotated.apply(extract_tags, axis=1)  
    
    #df_source['skills'] = df_source['skills'].apply(clean_skill_list,args=(skills, nlp, 0.9))
    

    try:
        os.remove(output_file)
    except:
        pass

    df_source.to_csv(output_file,sep=delimiter,index=False, encoding='utf-8')
    print('File', output_file, 'created successfully\n')

delimiter = '|'

nlp = spacy.load("en_core_web_lg")  # make sure to use larger package!

conn = Neo4jConnection(uri=os.getenv("DB_URI"), 
                    user=os.getenv("DB_USERNAME"),              
                    pwd=os.getenv("DB_PASSWORD"))


#extract_tags_from_annotation('indeed-annotated-DA-IT2.json', 'indeed-data-DA-IT2.csv', 'indeed-final-data-DA-IT2.csv', delimiter)
#extract_tags_from_annotation('livecareer-annotated-DA-IT2.json', 'livecareer-data-DA-IT2.csv', 'livecareer-final-data-DA-IT2.csv', delimiter)
#extract_tags_from_annotation('indeed-annotated-DE-IT2.json', 'indeed-data-DE-IT2.csv', 'indeed-final-data-DE-IT2.csv', delimiter)
#extract_tags_from_annotation('livecareer-annotated-DS-IT2.json', 'livecareer-data-DS-IT2.csv', 'livecareer-final-data-DS-IT2.csv', delimiter)
#extract_tags_from_annotation('livecareer-annotated-DE-IT2.json', 'livecareer-data-DE-IT2.csv', 'livecareer-final-data-DE-IT2.csv', delimiter)
#extract_tags_from_annotation('indeed-annotated-DS-IT2.json', 'indeed-data-DS-IT2.csv', 'indeed-final-data-DS-IT2.csv', delimiter)
extract_tags_from_annotation('vaia-data-NER.json', 'vaia-data-NER.csv', 'vaia-final-data-NER.csv', delimiter)

File vaia-final-data-NER.csv created successfully

